In [1]:
import os

In [2]:
pwd

'd:\\Personal\\Projects_Computer_Science_And_Engineering\\RaunakMLProjects\\End-to-end-ML-Project\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'd:\\Personal\\Projects_Computer_Science_And_Engineering\\RaunakMLProjects\\End-to-end-ML-Project'

In [24]:
from dataclasses import dataclass
from pathlib import Path
from typing import List
#entity
@dataclass(frozen=True)
class ModelEvaluationConfig:
    # Define the fields with type annotations
    root_dir: Path
    test_data_path: Path   #from confi
    model_path: Path    #from config
    all_params: dict
    metric_file_name: Path
    target_column: str
    selected_features: List[str]

  

In [25]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [26]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN
        selected_features = self.schema.SELECTED_FEATURES

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            selected_features = selected_features
           
        )

        return model_evaluation_config

In [27]:
import pandas as pd
import joblib
import os
from sklearn.metrics import accuracy_score
import numpy as np
from urllib.parse import urlparse
from src.mlProject.utils.common import save_json


In [28]:
# class ModelEvaluation:
#     def __init__(self, config: ModelEvaluationConfig):
#         self.config = config
    
#     def eval_metrics(self,actual, pred):
#         acc = accuracy_score(actual, pred)
#         return acc
        
#     def save_results(self):

#         test_data = pd.read_csv(self.config.test_data_path)
#         model = joblib.load(self.config.model_path)

#         test_x = test_data.drop([self.config.target_column], axis=1)
#         test_y = test_data[[self.config.target_column]]
        
#         predicted_qualities = model.predict(test_x)

#         acc = self.eval_metrics(test_y, predicted_qualities)
        
#         # Saving metrics as local
#         scores = {"Accuracy": acc}
#         save_json(path=Path(self.config.metric_file_name), data=scores)


In [29]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self,actual, pred):
        acc = accuracy_score(actual, pred)
        return acc
        
    def save_results(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data[self.config.selected_features]
        test_y = test_data[[self.config.target_column]]
        
        predicted_qualities = model.predict(test_x)

        acc = self.eval_metrics(test_y, predicted_qualities)
        
        # Saving metrics as local
        scores = {"Accuracy": acc}
        save_json(path=Path(self.config.metric_file_name), data=scores)


In [30]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2025-01-01 10:59:29,645: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-01 10:59:29,660: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-01 10:59:29,664: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-01 10:59:29,667: INFO: common: created directory at: artifacts]
[2025-01-01 10:59:29,674: INFO: common: created directory at: artifacts/model_evaluation]
[2025-01-01 10:59:55,463: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
